In [ ]:
import gzip
import csv
import json
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('pobreza22.csv.gz', compression='gzip')

In [ ]:
len(data)

In [ ]:
for col in data.columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

data = data.dropna()

In [ ]:
data = data.reset_index()

In [ ]:
len(data)

In [ ]:
columnas_deseadas = [
    'folioviv', 'foliohog', 'numren', 'est_dis', 'upm', 'factor',
    'paren', 'plp_e', 'plp', 'ent', 'rururb', 'edad', 'sexo', 'alfabetism',
    'trabajo_mp', 'id_men', 'ins_ali', 'hli', 'segpop', 'atemed',
    'telefono', 'celular', 'tv_paga', 'conex_inte', 'tarjeta', 'tenencia', 'aire_acond', 'calefacc',
    'gas_alim_e', 'gas_tran_e', 'disp_agua', 'drenaje', 'disp_elect', 'tot_resid',
    'd_ejh_supe', 'd_ejh_prep', 'd_ejh_secu', 'd_ejh_prim', 'd_ejh_sine',
    'trab_men1214', 'trab_men1517',
    'n_sinco_1', 'n_sinco_4', 'n_sinco_7', 'n_sinco_2', 'n_sinco_5', 'n_sinco_8',
    'n_sinco_3', 'n_sinco_6', 'n_sinco_9'
]

In [ ]:
# Filtra solo las columnas disponibles para evitar errores si alguna falta
data = data[[col for col in columnas_deseadas if col in data.columns]].copy()

In [ ]:
# Diccionario que mapea el número de entidad a la región
ent_a_region = {
    1: "1",            # Aguascalientes
    2: "2",               # Baja California
    3: "2",               # Baja California Sur
    4: "3",                # Campeche
    5: "4",                # Coahuila de Zaragoza
    6: "5",             # Colima
    7: "6",               # Chiapas
    8: "2",               # Chihuahua
    9: "7",              # Ciudad de México
    10: "2",              # Durango
    11: "1",           # Guanajuato
    12: "6",              # Guerrero
    13: "8",               # Hidalgo
    14: "5",             # Jalisco
    15: "7",             # México (Estado de México)
    16: "5",             # Michoacán de Ocampo
    17: "7",             # Morelos
    18: "5",             # Nayarit
    19: "4",               # Nuevo León
    20: "6",              # Oaxaca
    21: "8",               # Puebla
    22: "1",           # Querétaro
    23: "3",               # Quintana Roo
    24: "1",           # San Luis Potosí
    25: "2",              # Sinaloa
    26: "2",              # Sonora
    27: "3",               # Tabasco
    28: "4",               # Tamaulipas
    29: "8",               # Tlaxcala
    30: "8",               # Veracruz de Ignacio de la Llave
    31: "3",               # Yucatán
    32: "1",           # Zacatecas
}

In [ ]:
data['región'] = data['ent'].map(ent_a_region)

In [ ]:
data = data.drop('ent', axis=1)

In [ ]:
# Lista de columnas a evaluar
cols_conectividad = ['telefono', 'celular', 'tv_paga', 'conex_inte']

# Crear la nueva columna: 1 si alguno de ellos es 1, si no 0
data['conectividad'] = data[cols_conectividad].eq(1).any(axis=1).astype(int)

In [ ]:
data

In [ ]:
data = data.drop(['telefono', 'celular', 'tv_paga', 'conex_inte'], axis=1)

In [ ]:
# Crear variable: 1 si tiene al menos agua o drenaje, 0 si no tiene ninguno
data['agua_drenaje'] = data[['disp_agua', 'drenaje']].eq(1).all(axis=1).astype(int)

In [ ]:
data = data.drop(['disp_agua', 'drenaje'], axis=1)

In [ ]:
data['neducativojefe'] = np.nan

# Aplicar condiciones en orden de menor a mayor nivel educativo
data.loc[data['d_ejh_sine'] == 1, 'neducativojefe'] = 1
data.loc[data['d_ejh_prim'] == 1, 'neducativojefe'] = 2
data.loc[data['d_ejh_secu'] == 1, 'neducativojefe'] = 3
data.loc[data['d_ejh_prep'] == 1, 'neducativojefe'] = 4
data.loc[data['d_ejh_supe'] == 1, 'neducativojefe'] = 5

# Convertir a entero (opcional, si no esperas valores faltantes)
data['neducativojefe'] = data['neducativojefe'].astype('Int64')

In [ ]:
data = data.drop(['d_ejh_sine','d_ejh_prim','d_ejh_secu','d_ejh_prep','d_ejh_supe'], axis=1)

In [ ]:
data['trabajomenores'] = ((data['trab_men1214'] != 0) | (data['trab_men1517'] != 0)).astype(int)

In [ ]:
data = data.drop(['trab_men1517','trab_men1214'], axis=1)

In [ ]:
# Paso 1: crear columna binaria si la persona es menor de 18
data['es_ninio'] = (data['edad'] < 18).astype(int)

# Paso 2: agrupar por hogar (folioviv) y contar cuántos niños hay
conteo_niños = data.groupby('folioviv')['es_ninio'].sum()

# Paso 3: asignar el conteo a cada fila del DataFrame original
data['niños'] = data['folioviv'].map(conteo_niños)

# (opcional) eliminar la columna auxiliar
data = data.drop(columns='es_ninio')

In [ ]:
# Lista de columnas de ocupación
cols_ocup = [
    'n_sinco_1', 'n_sinco_2', 'n_sinco_3',
    'n_sinco_4', 'n_sinco_5', 'n_sinco_6',
    'n_sinco_7', 'n_sinco_8', 'n_sinco_9'
]

# Función que aplica la lógica para cada fila
def ocupacion_principal(row):
    valores = row[cols_ocup]
    max_valor = valores.max()

    if (valores == max_valor).sum() == 1:
        # Si solo una columna tiene el valor máximo
        return valores.idxmax()
    else:
        # Hay empate
        return "más de una ocupación principal"

# Aplicar la función fila por fila
data['ocupacion_hogar'] = data.apply(ocupacion_principal, axis=1)

In [ ]:
# Mapeo de nombres a números
mapeo_ocupacion = {
    'n_sinco_1': 1,
    'n_sinco_2': 2,
    'n_sinco_3': 3,
    'n_sinco_4': 4,
    'n_sinco_5': 5,
    'n_sinco_6': 6,
    'n_sinco_7': 7,
    'n_sinco_8': 8,
    'n_sinco_9': 9,
    'más de una ocupación principal': 10
}

# Aplicar el reemplazo en la columna ocupacion_hogar
data['ocupacion_hogar'] = data['ocupacion_hogar'].replace(mapeo_ocupacion)

In [ ]:
data = data.drop(['n_sinco_1', 'n_sinco_2', 'n_sinco_3',
    'n_sinco_4', 'n_sinco_5', 'n_sinco_6',
    'n_sinco_7', 'n_sinco_8', 'n_sinco_9'], axis=1)

In [ ]:
# 3. Filtrar para quedarte solo con paren == 1
data = data[data['paren'] == 1].copy()

In [ ]:
len(data)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
# 3. Filtrar para quedarte solo con numren == 1
data = data[data['numren'] == 1].copy()

In [ ]:
len(data)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data = data.drop('paren', axis=1)

In [ ]:
# Sumar los dos componentes del gasto estimado
df['gasto_total_e'] = df['gas_alim_e'] + df['gas_tran_e']

# Tomar el logaritmo natural del total
# Para evitar errores por ceros o valores negativos, puedes usar una transformación segura
df['consumption_expenditure'] = np.where(
    df['gasto_total_e'].isna() | (df['gasto_total_e'] == 0),
    0,
    np.log(df['gasto_total_e'])
)

In [ ]:
data = data.drop(['gas_alim_e','gas_tran_e'], axis=1)

In [ ]:
data = data.drop('id_men', axis=1)

In [ ]:
df['basic_energy'] = ((df['aire_acond'] == 1) | (df['calefacc'] == 1)).astype(int)

In [ ]:
data = data.drop(['aire_acond','calefacc'], axis=1)

In [ ]:
df['housing_tenure'] = (df['tenencia'] == 4).astype(int)

In [ ]:
data = data.drop('tenencia', axis=1)

In [ ]:
dfregresiones = data

In [ ]:
dfregresiones.columns

In [ ]:
import openpyxl

In [ ]:
dfregresiones.to_excel("dfregresion22.xlsx")

In [ ]:
dfML = data.drop(columns=['folioviv', 'foliohog', 'numren', 'est_dis', 'upm', 'factor'], axis=1)

In [ ]:
dfML.columns

In [ ]:
dfMLplp = dfML.drop('plp_e', axis=1)

In [ ]:
dfMLplp.columns #19 predictores y 1 variable independiente: plp

In [ ]:
dfMLplpe = dfML.drop('plp', axis=1)

In [ ]:
dfMLplpe.columns #19 predictores y 1 variable independiente: plp_e

In [ ]:
dfML.to_excel("dfML22.xlsx")

In [ ]:
dfMLplp.to_excel("dfMLplp22.xlsx")

In [ ]:
dfMLplpe.to_excel("dfMLplpe22.xlsx")